<a href="https://colab.research.google.com/github/lover-swan/project2_2/blob/main/pretreatment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
data=pd.read_pickle('/content/drive/MyDrive/project2_2/data.pkl')

In [ ]:
data

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,36,541312140
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33,554748717
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543,519107250
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,252,550050854
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1082,535871217
...,...,...,...,...,...,...,...,...
41705910,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527,537931532
41705911,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,NaN,redmond,62,527322328
41705912,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,129,566280422
41705913,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,electronics.clocks,tissot,690,513118352


In [ ]:
# 클릭을 0으로, 장바구니를 1로 하는 target 값 생성성

In [ ]:
data['target']=np.where(data['event_type']!='view',1,0)

In [ ]:
data=data.drop(columns='event_type')

In [ ]:
data.target=data.target.astype('int8')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41705915 entries, 0 to 41705914
Data columns (total 8 columns):
 #   Column         Dtype   
---  ------         -----   
 0   event_time     object  
 1   product_id     int32   
 2   category_id    int64   
 3   category_code  object  
 4   brand          category
 5   price          int16   
 6   user_id        int32   
 7   target         int8    
dtypes: category(1), int16(1), int32(2), int64(1), int8(1), object(2)
memory usage: 1.4+ GB


In [ ]:
# 모델링을 위해 unix 시간으로 변환환

In [ ]:
data['event_time']=data['event_time'].str[:-4]

In [ ]:
data['event_time']=pd.to_datetime(data['event_time'])

In [ ]:
data['event_time']=data['event_time'].astype('int64')

<ipython-input-3-e6e855d2dbce>:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  data['event_time']=data['event_time'].astype('int64')


In [ ]:
data['event_time']=data['event_time'].apply(lambda x: x/1000000000)

In [ ]:
data['event_time']=data['event_time'].astype('int32')

In [15]:
#결측치 처리

In [42]:
data

,event_time,product_id,category_id,category_code,brand,price,user_id,target
0,1569888000,44600062,2103807459595387724,nan,shiseido,36,541312140,0
1,1569888000,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33,554748717,0
2,1569888001,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543,519107250,0
3,1569888001,1307067,2053013558920217191,computers.notebook,lenovo,252,550050854,0
4,1569888004,1004237,2053013555631882655,electronics.smartphone,apple,1082,535871217,0
...,...,...,...,...,...,...,...,...
41705910,1572566398,2300275,2053013560530830019,electronics.camera.video,gopro,527,537931532,0
41705911,1572566398,10800172,2053013554994348409,nan,redmond,62,527322328,0
41705912,1572566398,5701038,2053013553970938175,auto.accessories.player,kenwood,129,566280422,0
41705913,1572566399,21407424,2053013561579406073,electronics.clocks,tissot,690,513118352,0


In [16]:
data.isna().sum()

event_time              0
product_id              0
category_id             0
category_code    13342184
brand             6058775
price                   0
user_id                 0
target                  0
dtype: int64

In [41]:
data.category_code=data.category_code.astype('str')

In [43]:
def get_nan(df):
  value={}
  for i,k in zip(df['category_id'],df['category_code']):
    if k != 'nan':
      value[i]=k
  return value

In [44]:
category_list=get_nan(data)

In [78]:
def fill_nan(df):
  category=df['category_code'].to_numpy()
  for i,k in tqdm(enumerate(df['category_id'].to_numpy())):
    if k in category_list.keys():
      category[i]=category_list[k]

  return category

In [79]:
data['category_code']=fill_nan(data)

41705915it [00:28, 1460487.34it/s]


In [80]:
data

,event_time,product_id,category_id,category_code,brand,price,user_id,target
0,1569888000,44600062,NaN,NaN,shiseido,36,541312140,0
1,1569888000,3900821,appliances.environment.water_heater,appliances.environment.water_heater,aqua,33,554748717,0
2,1569888001,17200506,furniture.living_room.sofa,furniture.living_room.sofa,NaN,543,519107250,0
3,1569888001,1307067,computers.notebook,computers.notebook,lenovo,252,550050854,0
4,1569888004,1004237,electronics.smartphone,electronics.smartphone,apple,1082,535871217,0
...,...,...,...,...,...,...,...,...
41705910,1572566398,2300275,electronics.camera.video,electronics.camera.video,gopro,527,537931532,0
41705911,1572566398,10800172,NaN,NaN,redmond,62,527322328,0
41705912,1572566398,5701038,auto.accessories.player,auto.accessories.player,kenwood,129,566280422,0
41705913,1572566399,21407424,electronics.clocks,electronics.clocks,tissot,690,513118352,0


In [66]:
data.category_code=data.category_code.astype('category')

In [73]:
data=data.replace('nan',np.nan)

In [74]:
data.isna().sum()

event_time              0
product_id              0
category_id             0
category_code    13342184
brand             6058775
price                   0
user_id                 0
target                  0
dtype: int64

In [ ]:
data.to_pickle('/content/drive/MyDrive/project2_2/data.pkl')

In [ ]:
data1=data[:7000000]
data2=data[7000000:14000000]
data3=data[14000000:21000000]
data4=data[21000000:]

In [ ]:
data1.to_pickle('/content/drive/MyDrive/project2_2/data1_1.pkl')
data2.to_pickle('/content/drive/MyDrive/project2_2/data1_2.pkl')
data3.to_pickle('/content/drive/MyDrive/project2_2/data1_3.pkl')
data4.to_pickle('/content/drive/MyDrive/project2_2/data1_4.pkl')